In [1]:
from set_math import *
from datasets_init import *

sympy imported
pandas imported
numpy imported
scipy imported


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


statsmodels imported
statsmodels.formula imported
statasmodels.stats.api imported
train.csv good
test.csv good
you may now begin


# Basic Features

In [2]:
# 반품 발생은 0으로 표기하는 컬럼 추가
train['Count'] = train['ScanCount']
train['Count'][train['ScanCount']<0] = 0
train['Count'].head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0    0
1    1
2    1
3    2
4    2
Name: Count, dtype: int64

In [3]:
# Fineline 없는 것은 9999로 명시
train['FinelineNumber'].fillna(value=9999, inplace=True)

# Upc 없는 것은 -1로 명시
train['Upc'].fillna(value=-1, inplace=True)


train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647054 entries, 0 to 647053
Data columns (total 8 columns):
TripType                 647054 non-null int64
VisitNumber              647054 non-null int64
Weekday                  647054 non-null object
Upc                      647054 non-null float64
ScanCount                647054 non-null int64
DepartmentDescription    645693 non-null object
FinelineNumber           647054 non-null float64
Count                    647054 non-null int64
dtypes: float64(2), int64(4), object(2)
memory usage: 39.5+ MB


In [4]:
# Department 인코딩  (X)
from sklearn import preprocessing
train['DepartmentDescription'] = preprocessing.LabelEncoder().fit_transform(list(train['DepartmentDescription']))
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count
0,999,5,Friday,6.811315e+10,-1,20,1000.0,0
1,30,7,Friday,6.053882e+10,1,62,8931.0,1
2,30,7,Friday,7.410811e+09,1,50,4504.0,1
3,26,8,Friday,2.238404e+09,2,49,3565.0,2
4,26,8,Friday,2.006614e+09,2,49,1017.0,2


In [5]:
# Weekday 인코딩
train['Weekday'] = preprocessing.LabelEncoder().fit_transform(list(train['Weekday']))
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count
0,999,5,0,6.811315e+10,-1,20,1000.0,0
1,30,7,0,6.053882e+10,1,62,8931.0,1
2,30,7,0,7.410811e+09,1,50,4504.0,1
3,26,8,0,2.238404e+09,2,49,3565.0,2
4,26,8,0,2.006614e+09,2,49,1017.0,2


In [6]:
# VisitNumber, DepartmentDescription으로 groupby. ScanCount sum(총 판매량) 
df = train.groupby(['VisitNumber', 'DepartmentDescription'], as_index=False)['Count'].sum()
df.head()

,VisitNumber,DepartmentDescription,Count
0,5,20,0
1,7,50,1
2,7,62,1
3,8,16,2
4,8,17,1


In [7]:
df1 = df.groupby(['VisitNumber'], as_index=False)['Count'].min()
df1.rename(columns={'Count': 'Min'}, inplace=True)
df1.head()

,VisitNumber,Min
0,5,0
1,7,1
2,8,1
3,9,1
4,10,1


In [8]:
df2 = df.groupby(['VisitNumber'], as_index=False)['Count'].max()
df2.rename(columns={'Count': 'Max'}, inplace=True)
df2.head()

,VisitNumber,Max
0,5,0
1,7,1
2,8,20
3,9,2
4,10,2


In [9]:
df3 = df.groupby(['VisitNumber'], as_index=False)['Count'].mean()
df3.rename(columns={'Count': 'Mean'}, inplace=True)
df3.head()

,VisitNumber,Mean
0,5,0.000000
1,7,1.000000
2,8,4.285714
3,9,1.500000
4,10,1.500000


In [10]:
train = train.merge(df1, how='left', on=['VisitNumber'], copy=True)
train = train.merge(df2, how='left', on=['VisitNumber'], copy=True)
train = train.merge(df3, how='left', on=['VisitNumber'], copy=True)  
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,Mean
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,0.000000
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,1.000000
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,1.000000
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,4.285714
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,4.285714


In [11]:
train['Range'] = train['Max'] - train['Min']
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,Mean,Range
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,0.000000,0
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,1.000000,0
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,1.000000,0
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,4.285714,19
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,4.285714,19


In [12]:
df = train[train['DepartmentDescription']==67]   #????????
df.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,Mean,Range
316,24,140,0,7.350780e+10,4,67,1710.0,4,1,25,7.166667,24
401,31,209,0,6.811310e+10,1,67,9998.0,1,2,2,2.000000,0
402,31,209,0,7.993666e+10,1,67,765.0,1,2,2,2.000000,0
650,31,288,0,7.993663e+10,1,67,1065.0,1,1,1,1.000000,0
1535,31,666,0,8.859131e+10,1,67,170.0,1,1,4,2.500000,3


In [13]:
df = df.groupby(['VisitNumber'], as_index=False)['Count'].count()
df.head()

,VisitNumber,Count
0,140,1
1,209,2
2,288,1
3,666,1
4,724,2


In [14]:
df.rename(columns={'Count': 'Null'}, inplace=True)
df.head()

,VisitNumber,Null
0,140,1
1,209,2
2,288,1
3,666,1
4,724,2


In [15]:
train = train.merge(df, how='left', on=['VisitNumber'], copy=True) 
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,Mean,Range,Null
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,0.000000,0,NaN
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,1.000000,0,NaN
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,1.000000,0,NaN
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,4.285714,19,NaN
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,4.285714,19,NaN


In [16]:
train['Null'].fillna(value=0, inplace=True) 
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,Mean,Range,Null
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,0.000000,0,0.0
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,1.000000,0,0.0
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,1.000000,0,0.0
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,4.285714,19,0.0
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,4.285714,19,0.0


In [17]:
train['Null'][train['Null']>0] = 1
train.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,Mean,Range,Null
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,0.000000,0,0.0
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,1.000000,0,0.0
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,1.000000,0,0.0
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,4.285714,19,0.0
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,4.285714,19,0.0


In [18]:
df = train[train['ScanCount']<0]
df = df.groupby(['VisitNumber'], as_index=False)['Count'].count()

df.head()

,VisitNumber,Count
0,5,1
1,8,2
2,132,2
3,133,1
4,182,1


In [19]:
df.rename(columns={'Count': 'Has_Neg'}, inplace=True) 
train = train.merge(df, how='left', on=['VisitNumber'], copy=True)

train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,Mean,Range,Null,Has_Neg
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,0.000000,0,0.0,1.0
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,1.000000,0,0.0,NaN
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,1.000000,0,0.0,NaN
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,4.285714,19,0.0,2.0
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,4.285714,19,0.0,2.0


In [20]:
train['Has_Neg'].fillna(value=0, inplace=True)
train['Has_Neg'][train['Has_Neg']>0] = 1

train.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,Mean,Range,Null,Has_Neg
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,0.000000,0,0.0,1.0
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,1.000000,0,0.0,0.0
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,1.000000,0,0.0,0.0
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,4.285714,19,0.0,1.0
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,4.285714,19,0.0,1.0


In [21]:
df = train[train['FinelineNumber']==9999]
df.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,Mean,Range,Null,Has_Neg
25,26,8,0,-1.0,1,68,9999.0,1,1,20,4.285714,19,0.0,1.0
548,27,259,0,-1.0,3,68,9999.0,3,1,10,5.000000,9,0.0,0.0
549,27,259,0,-1.0,1,68,9999.0,1,1,10,5.000000,9,0.0,0.0
959,999,409,0,-1.0,-1,68,9999.0,0,0,0,0.000000,0,0.0,1.0
1116,39,479,0,-1.0,1,68,9999.0,1,1,3,1.666667,2,0.0,0.0


In [22]:
df = df.groupby(['VisitNumber'], as_index=False)['Count'].count()
df.head()

,VisitNumber,Count
0,8,1
1,259,2
2,409,1
3,479,1
4,484,2


In [23]:
df.rename(columns={'Count': 'Missing'}, inplace=True)
df.head()

,VisitNumber,Missing
0,8,1
1,259,2
2,409,1
3,479,1
4,484,2


In [24]:
train = train.merge(df, how='left', on=['VisitNumber'], copy=True) 
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,Mean,Range,Null,Has_Neg,Missing
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,0.000000,0,0.0,1.0,NaN
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,1.000000,0,0.0,0.0,NaN
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,1.000000,0,0.0,0.0,NaN
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,4.285714,19,0.0,1.0,1.0
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,4.285714,19,0.0,1.0,1.0


In [25]:
train['Missing'].fillna(value=0, inplace=True)
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,Mean,Range,Null,Has_Neg,Missing
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,0.000000,0,0.0,1.0,0.0
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,1.000000,0,0.0,0.0,0.0
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,1.000000,0,0.0,0.0,0.0
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,4.285714,19,0.0,1.0,1.0
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,4.285714,19,0.0,1.0,1.0


In [26]:
train['Missing'][train['Missing']>0] = 1
train.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,Mean,Range,Null,Has_Neg,Missing
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,0.000000,0,0.0,1.0,0.0
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,1.000000,0,0.0,0.0,0.0
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,1.000000,0,0.0,0.0,0.0
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,4.285714,19,0.0,1.0,1.0
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,4.285714,19,0.0,1.0,1.0


In [27]:
df = train.groupby(['VisitNumber',  'FinelineNumber'], as_index=False)['Count'].count()
df.head()

,VisitNumber,FinelineNumber,Count
0,5,1000.0,1
1,7,4504.0,1
2,7,8931.0,1
3,8,7.0,1
4,8,707.0,1


In [28]:
df = df.groupby(['VisitNumber'], as_index=False)['Count'].count()
df.head()

,VisitNumber,Count
0,5,1
1,7,2
2,8,17
3,9,3
4,10,3


In [29]:
df.rename(columns={'Count': 'N_Fineline'}, inplace=True) 
df.head()

,VisitNumber,N_Fineline
0,5,1
1,7,2
2,8,17
3,9,3
4,10,3


In [30]:
train = train.merge(df, how='left', on=['VisitNumber'], copy=True)
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,Mean,Range,Null,Has_Neg,Missing,N_Fineline
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,0.000000,0,0.0,1.0,0.0,1
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,1.000000,0,0.0,0.0,0.0,2
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,1.000000,0,0.0,0.0,0.0,2
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,4.285714,19,0.0,1.0,1.0,17
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,4.285714,19,0.0,1.0,1.0,17


In [31]:
df = train.groupby(['VisitNumber',  'Upc'], as_index=False)['Count'].count()
df.head()

,VisitNumber,Upc,Count
0,5,6.811315e+10,1
1,7,7.410811e+09,1
2,7,6.053882e+10,1
3,8,-1.000000e+00,1
4,8,2.006614e+09,1


In [32]:
df = df.groupby(['VisitNumber'], as_index=False)['Count'].count()
df.head()

,VisitNumber,Count
0,5,1
1,7,2
2,8,21
3,9,3
4,10,3


In [33]:
df.rename(columns={'Count': 'N_Upc'}, inplace=True) 
df.head()

,VisitNumber,N_Upc
0,5,1
1,7,2
2,8,21
3,9,3
4,10,3


In [34]:
train = train.merge(df, how='left', on=['VisitNumber'], copy=True) 
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,Mean,Range,Null,Has_Neg,Missing,N_Fineline,N_Upc
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,0.000000,0,0.0,1.0,0.0,1,1
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,1.000000,0,0.0,0.0,0.0,2,2
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,1.000000,0,0.0,0.0,0.0,2,2
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,4.285714,19,0.0,1.0,1.0,17,21
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,4.285714,19,0.0,1.0,1.0,17,21


In [35]:
df = train.groupby(['VisitNumber', 'DepartmentDescription'], as_index=False)['Count'].count()
df.head()

,VisitNumber,DepartmentDescription,Count
0,5,20,1
1,7,50,1
2,7,62,1
3,8,16,1
4,8,17,1


In [36]:
df = df.groupby(['VisitNumber'], as_index=False)['Count'].count()
df.head()

,VisitNumber,Count
0,5,1
1,7,2
2,8,7
3,9,2
4,10,2


In [37]:
df.rename(columns={'Count': 'N_Dep'}, inplace=True) 
df.head()

,VisitNumber,N_Dep
0,5,1
1,7,2
2,8,7
3,9,2
4,10,2


In [38]:
train = train.merge(df, how='left', on=['VisitNumber'], copy=True)
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,Mean,Range,Null,Has_Neg,Missing,N_Fineline,N_Upc,N_Dep
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,0.000000,0,0.0,1.0,0.0,1,1,1
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,1.000000,0,0.0,0.0,0.0,2,2,2
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,1.000000,0,0.0,0.0,0.0,2,2,2
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,4.285714,19,0.0,1.0,1.0,17,21,7
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,4.285714,19,0.0,1.0,1.0,17,21,7


In [39]:
df = train.groupby(['VisitNumber'], as_index=False)['Count'].sum()
df.head()

,VisitNumber,Count
0,5,0
1,7,2
2,8,30
3,9,3
4,10,3


In [40]:
df.rename(columns={'Count': 'Sum'}, inplace=True)
df.head()

,VisitNumber,Sum
0,5,0
1,7,2
2,8,30
3,9,3
4,10,3


In [41]:
train = train.merge(df, how='left', on=['VisitNumber'], copy=True)
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,Mean,Range,Null,Has_Neg,Missing,N_Fineline,N_Upc,N_Dep,Sum
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,0.000000,0,0.0,1.0,0.0,1,1,1,0
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,1.000000,0,0.0,0.0,0.0,2,2,2,2
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,1.000000,0,0.0,0.0,0.0,2,2,2,2
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,4.285714,19,0.0,1.0,1.0,17,21,7,30
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,4.285714,19,0.0,1.0,1.0,17,21,7,30


In [42]:
train['Ratio_F_D'] = train['N_Fineline']/train['N_Dep']
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,Mean,Range,Null,Has_Neg,Missing,N_Fineline,N_Upc,N_Dep,Sum,Ratio_F_D
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,0.000000,0,0.0,1.0,0.0,1,1,1,0,1.000000
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,1.000000,0,0.0,0.0,0.0,2,2,2,2,1.000000
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,1.000000,0,0.0,0.0,0.0,2,2,2,2,1.000000
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,4.285714,19,0.0,1.0,1.0,17,21,7,30,2.428571
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,4.285714,19,0.0,1.0,1.0,17,21,7,30,2.428571


In [43]:
train['Ratio_U_D'] = train['N_Upc']/train['N_Dep'] 
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,...,Range,Null,Has_Neg,Missing,N_Fineline,N_Upc,N_Dep,Sum,Ratio_F_D,Ratio_U_D
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,...,0,0.0,1.0,0.0,1,1,1,0,1.000000,1.0
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,...,0,0.0,0.0,0.0,2,2,2,2,1.000000,1.0
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,...,0,0.0,0.0,0.0,2,2,2,2,1.000000,1.0
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,...,19,0.0,1.0,1.0,17,21,7,30,2.428571,3.0
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,...,19,0.0,1.0,1.0,17,21,7,30,2.428571,3.0


In [44]:
train['mean_to_min'] = train['Mean']/train['Min']
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,...,Null,Has_Neg,Missing,N_Fineline,N_Upc,N_Dep,Sum,Ratio_F_D,Ratio_U_D,mean_to_min
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,...,0.0,1.0,0.0,1,1,1,0,1.000000,1.0,NaN
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,...,0.0,0.0,0.0,2,2,2,2,1.000000,1.0,1.000000
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,...,0.0,0.0,0.0,2,2,2,2,1.000000,1.0,1.000000
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,...,0.0,1.0,1.0,17,21,7,30,2.428571,3.0,4.285714
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,...,0.0,1.0,1.0,17,21,7,30,2.428571,3.0,4.285714


In [45]:
train['mean_to_min'][train['Min']==0] = 0
train.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,...,Null,Has_Neg,Missing,N_Fineline,N_Upc,N_Dep,Sum,Ratio_F_D,Ratio_U_D,mean_to_min
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,...,0.0,1.0,0.0,1,1,1,0,1.000000,1.0,0.000000
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,...,0.0,0.0,0.0,2,2,2,2,1.000000,1.0,1.000000
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,...,0.0,0.0,0.0,2,2,2,2,1.000000,1.0,1.000000
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,...,0.0,1.0,1.0,17,21,7,30,2.428571,3.0,4.285714
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,...,0.0,1.0,1.0,17,21,7,30,2.428571,3.0,4.285714


In [46]:
train['max_to_mean'] = train['Max']/train['Mean']
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,...,Has_Neg,Missing,N_Fineline,N_Upc,N_Dep,Sum,Ratio_F_D,Ratio_U_D,mean_to_min,max_to_mean
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,...,1.0,0.0,1,1,1,0,1.000000,1.0,0.000000,NaN
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,...,0.0,0.0,2,2,2,2,1.000000,1.0,1.000000,1.000000
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,...,0.0,0.0,2,2,2,2,1.000000,1.0,1.000000,1.000000
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,...,1.0,1.0,17,21,7,30,2.428571,3.0,4.285714,4.666667
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,...,1.0,1.0,17,21,7,30,2.428571,3.0,4.285714,4.666667


In [47]:
train['max_to_mean'][train['Mean']==0] = 0    
train.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,...,Has_Neg,Missing,N_Fineline,N_Upc,N_Dep,Sum,Ratio_F_D,Ratio_U_D,mean_to_min,max_to_mean
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,...,1.0,0.0,1,1,1,0,1.000000,1.0,0.000000,0.000000
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,...,0.0,0.0,2,2,2,2,1.000000,1.0,1.000000,1.000000
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,...,0.0,0.0,2,2,2,2,1.000000,1.0,1.000000,1.000000
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,...,1.0,1.0,17,21,7,30,2.428571,3.0,4.285714,4.666667
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,...,1.0,1.0,17,21,7,30,2.428571,3.0,4.285714,4.666667


### Basic Feature 요약

- 

* * * *

# Department_Count_Product

In [48]:
dd_dummies = pd.get_dummies(train.DepartmentDescription)
dd_dummies.shape

(647054, 69)

In [49]:
dd_dummies_2 = dd_dummies.copy()

In [50]:
for i in range(dd_dummies.shape[1]):
    dd_dummies_2['Ratio_%s' % (dd_dummies_2.columns[i])] = dd_dummies_2[i]/train['Sum']

#sum이 0이면 NaN이 발생한다
dd_dummies_2.fillna(value=0, inplace=True)

In [51]:
dd_dummies_ratio = dd_dummies_2.iloc[:, 69:]
dd_dummies_ratio.shape

(647054, 69)

### DepartmentDescription_Count

- DepartmentDescription을 모두 Dummy 변수로 추가하는 내용. (get_dummy로 대체 가능)
- VisitNumber별로 발생한 각 DepartmentDescription이, Visitnumber의 전체 ScanCount(판매량)에서 차지하는 비율 ( Ratio_$i$ ) 추가

* * * *

# Department_Count_Neg

In [52]:
dd_dummies_2 = dd_dummies.copy()

In [53]:
for i in range(69):
    dd_dummies_2['Return_%s' % (dd_dummies_2.columns[i])] = dd_dummies_2[i]*train['ScanCount'][train['ScanCount'] < 0]
    
dd_dummies_2.fillna(value=0, inplace=True)

In [54]:
dd_dummies_return_count = dd_dummies_2.iloc[:, 69:]
dd_dummies_return_count.shape

(647054, 69)

* * * *

# Department_Counts_Multiple_Products

In [55]:
dd_dummies_2 = dd_dummies.copy()

In [56]:
for i in range(69):
    dd_dummies_2['Multi_Count_%s' % (dd_dummies_2.columns[i])] = dd_dummies_2[i]*train['ScanCount'][train['ScanCount'] > 1 ]
    
dd_dummies_2.fillna(value=0, inplace=True)

In [57]:
dd_dummies_multi_count = dd_dummies_2.iloc[:, 69:]
dd_dummies_multi_count.shape

(647054, 69)

# Department_Counts_Multiple_Rows 
### (각 DD별로 VisitType 당  몇 번 등장하는지 더미변수 생성, 구매/반품 상관 X)

# Department_Counts_Rows_Pos_ScanCount

### (구매를 위한 Scan을 몇 번 했는지, 각 DD별로 더미변수 생성)

# Department_Counts_Rows_Neg_Scancount

### (반품을 위한 Scan을 몇 번 했는지, 각 DD별로 더미변수 생성)

* * * *

# Fineline_Count 

- Fineline 더미변수 추가

In [58]:
train.shape

(647054, 23)

In [59]:
fine_dummies  = pd.get_dummies(train.FinelineNumber)
fine_dummies.shape

(647054, 5196)

### Fineline Count 요약
- Fineline을 모두 Dummy 변수로 추가하는 내용. (get_dummy로 대체 가능)

In [60]:
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,...,Has_Neg,Missing,N_Fineline,N_Upc,N_Dep,Sum,Ratio_F_D,Ratio_U_D,mean_to_min,max_to_mean
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,...,1.0,0.0,1,1,1,0,1.000000,1.0,0.000000,0.000000
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,...,0.0,0.0,2,2,2,2,1.000000,1.0,1.000000,1.000000
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,...,0.0,0.0,2,2,2,2,1.000000,1.0,1.000000,1.000000
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,...,1.0,1.0,17,21,7,30,2.428571,3.0,4.285714,4.666667
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,...,1.0,1.0,17,21,7,30,2.428571,3.0,4.285714,4.666667


In [61]:
train = pd.concat([train, dd_dummies_ratio], axis=1)
train = pd.concat([train, dd_dummies_return_count], axis=1)
train = pd.concat([train, dd_dummies_multi_count], axis=1)
train = pd.concat([train, fine_dummies], axis=1)

print(train.shape)
train.head()

(647054, 5426)


,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Min,Max,...,9966.0,9967.0,9970.0,9971.0,9974.0,9975.0,9991.0,9997.0,9998.0,9999.0
0,999,5,0,6.811315e+10,-1,20,1000.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30,7,0,6.053882e+10,1,62,8931.0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,30,7,0,7.410811e+09,1,50,4504.0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,26,8,0,2.238404e+09,2,49,3565.0,2,1,20,...,0,0,0,0,0,0,0,0,0,0
4,26,8,0,2.006614e+09,2,49,1017.0,2,1,20,...,0,0,0,0,0,0,0,0,0,0


In [64]:
train_2 = train.copy()

In [ ]:
# gb1 = train.groupby(['VisitNumber']).sum()

In [ ]:
# gb1

# Department_Dummies (Merge 하는 과정 추정... 아직 모르겠음)

# 다음과 같이 추리고 나서 시도 해볼 수 있음

- Product counts for 70% most frequent Fineline products
- Department dummies and their two-way interactions (only for 60% most frequent departments)

# Train Data에 적용

- "HEALTH AND BEAUTY AIDS" 드랍